# Distance Validation

The goal here is to compare the distances output by our algorithm to some known distances to try to validate the former. We'll start with the series of images presented in Figure 9 of _Chiodi et al., 2021_:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from cv_experiments.seg2info import Seg2Info
from datetime import datetime, timezone

s2i = Seg2Info(output_props={"width": 1920, "height": 1072})

images = s2i.load_dirs("../validation/fig9_seginput", "../validation/fig9_segmaps")
images = images[:9]  # The images after this are just the boat being stuck in ice
def fig9title(image):
    dt = datetime.fromtimestamp(int(image['name'].split('.')[0].split('_')[2]), tz=timezone.utc)
    return f"{dt:%H:%M:%S}"
print([image["name"] for image in images])
print(images[0]["segmap"].shape)
s2i.plot_all(images, lambda ax, image: s2i.simple_composite(ax, image, title=fig9title(image)), lambda fig: fig.subplots_adjust(hspace=-0.5))
s2i.plot_all(images, lambda ax, image: s2i.plot_mask(ax, image, title=fig9title(image)), lambda fig: fig.subplots_adjust(hspace=-0.5))

Converted to log-polar, that's:

In [ ]:
s2i.apply(s2i.whole_pipeline, images, "segmap", "logpolar_final")
s2i.plot_all(images, lambda ax, image: s2i.plot_log_polar(ax, image, "logpolar_final", title=fig9title(image)), lambda fig: fig.subplots_adjust(hspace=0.5))

The idea here is that the boat gets stuck in the ice at around 00:39:30, which lets us estimate the ice's velocity by the boat's speed over the next few minutes while it is stuck in the ice. Then, we can extrapolate backwards to find the ice's position at a given time (assuming it moves at constant velocity), which lets us calculate the distance from the boat to the ice for the sequence of images above.

Let's get a table of lat/lons for the image timestamps, a table of speeds and headings while stuck in ice, an estimate of the collision time, and a known ice lat/lon and time.

In [ ]:
import pandas as pd
latlon_data = pd.read_csv("../validation/fig9_latlons.csv", index_col="time")
stuck_data = pd.read_csv("../validation/fig9_stuck_SOG_COG.csv", index_col="time")
params_data = pd.read_csv("../validation/fig9_params.csv", index_col="key")
display(latlon_data)
display(stuck_data)
display(params_data)

Okay, time for some math.

In [ ]:
import numpy as np
from geographiclib.geodesic import Geodesic
import matplotlib.pyplot as plt

# Interesting discussion at https://math.stackexchange.com/questions/3321142/how-to-compute-the-average-in-modular-arithmetic
def modular_avg(data, mod):
    TAU = 2*np.pi
    remapped = data*TAU/mod
    points = np.array([np.sin(remapped), np.cos(remapped)])
    avg_yx = points.mean(axis=1)
    mag = np.hypot(*avg_yx)
    if mag < 0.5: print(f"Modular average risks being meaningless (magnitude={mag})")
    result = np.arctan2(*avg_yx)*mod/TAU
    return result

def polar_to_vector(mag, angle_deg):
    """Convert a magnitude and an angle in degrees (measured clockwise with 0 as north) to an x,y vector"""
    angle_rad = -np.deg2rad(angle_deg)+np.pi/2  # Compass to unit circle
    uv = np.array([np.cos(angle_rad), np.sin(angle_rad)])
    return uv*mag

def sog_cog_to_ms(sog_kts, cog_deg):
    """ Convert speed over ground in knots and course over ground in degrees to an x,y vector in m/s
    :param sog_kts: speed over ground in knots
    :param cog_deg: course over ground in degrees, measured clockwise with 0 as north
    :returns: an x,y vector in m/s where +x is east, +y is north
    """
    MS_PER_KT = 0.514
    return polar_to_vector(sog_kts*MS_PER_KT, cog_deg)

def latlon_offset_to_m(src_latlon, dest_latlon):
    """ Find a vector representing the shortest path over the Earth from the source lat/lon to the destination lat/lon
    :param src_latlon: a lat,lon pair representing the source
    :param dest_latlon: a lat,lon pair representing the destination
    :returns: an x,y vector in meters where +x is east, +y is north
    """

    geod = Geodesic.WGS84
    path = geod.Inverse(*src_latlon, *dest_latlon)
    dist = path["s12"]
    # We are given two azimuths of the line: one from the perspective of the start point and one the end point
    # Let's just average them
    course = np.mean([path["azi1"], path["azi2"]])
    return polar_to_vector(dist, course)

origin_lat_lon = np.array([params_data["value"]["lat_basis"], params_data["value"]["lon_basis"]])
origin_time = params_data["value"]["t_basis"]

avg_ice_speed = stuck_data["LTF SOG"].mean()
avg_ice_course = modular_avg(stuck_data["LTF COG"], 360)
ice_vector = sog_cog_to_ms(avg_ice_speed, avg_ice_course)

times = np.array(latlon_data.index)
ice_positions = np.outer((times-origin_time)*60, ice_vector)
boat_positions = np.array([latlon_offset_to_m(origin_lat_lon, x[1].values)
    for x in latlon_data[["lat", "lon"]].iterrows()])

plt.figure(figsize=(6,6))
plt.gca().axis("equal")
plt.plot(*(ice_positions.T))
plt.plot(*(boat_positions.T))
plt.show()

distances = np.hypot(*(ice_positions-boat_positions).T)
plt.scatter(times, distances)
print(distances)

This is… okay, but it strikes us now that it might be both simpler and more accurate to bypass speed measurements altogether and just pick a pair of points to extrapolate based on:

In [ ]:
ice_data = pd.read_csv("../validation/fig9_ice_refs.csv", index_col="time")
display(ice_data)
ice_vector = latlon_offset_to_m(ice_data.iloc[0][["lat", "lon"]].values, ice_data.iloc[1][["lat", "lon"]].values)/((ice_data.index[1]-ice_data.index[0])*60)
origin_lat_lon = ice_data.iloc[0][["lat", "lon"]].values
origin_time = ice_data.index[0]

ice_positions = np.outer((times-origin_time)*60, ice_vector)
boat_positions = np.array([latlon_offset_to_m(origin_lat_lon, x[1].values)
    for x in latlon_data[["lat", "lon"]].iterrows()])

plt.figure(figsize=(6,6))
plt.gca().axis("equal")
plt.plot(*(ice_positions.T))
plt.plot(*(boat_positions.T))
# plt.scatter(0, 0)
plt.show()

distances = np.hypot(*(ice_positions-boat_positions).T)
plt.scatter(times, distances)
print(np.stack([times, distances]).T)

Thus, we get quite different values depending on whether we account for ice movement or not:

In [ ]:
distances_naive = np.hypot(*(boat_positions).T)
plt.scatter(times, distances_naive, label="Without ice movement")
plt.scatter(times, distances, label="With ice movement")
plt.legend()

summary_table = pd.DataFrame({"time": times, "no_ice_mvt": distances_naive, "yes_ice_mvt": distances}).set_index("time")
with pd.option_context("display.precision", 2): display(summary_table)

Okay, now let's figure out what our images are telling us about the distance to the closest ice. Roughly, we will calculate the fraction of the field of view that is ice at every distance increment and report the closest one that has more than, say, 5% ice coverage. This rules out little "outlier" chunks of ice.

In [ ]:
def closest_ice(logpolar_plot, ice_thresh=0.05, nan_thresh=0.1):
    num = np.nansum(logpolar_plot, axis=1)
    n_nonnan = (~np.isnan(logpolar_plot)).sum(axis=1)
    frac_ice = np.where(n_nonnan/logpolar_plot.shape[1] > nan_thresh, np.divide(num, n_nonnan, where=(n_nonnan != 0)), np.nan)
    is_ice = frac_ice > ice_thresh
    i_last = logpolar_plot.shape[0]-np.argmax(is_ice[::-1])-1
    dist_last = s2i.y2dist(i_last)
    return dist_last

summary_table["camera_predicted"] = s2i.apply(closest_ice, images, "logpolar_final")

In [ ]:
with pd.option_context("display.precision", 2): display(summary_table)
plt.scatter(times, distances_naive, label="Without ice movement")
plt.scatter(times, distances, label="With ice movement")
plt.scatter(times, summary_table["camera_predicted"], label="Camera predicted nearest")
plt.xlabel("Minutes after the hour")
plt.ylabel("Distance to ice (m)")
plt.legend()

At a first glance… nope, it doesn't work.

However, there are many potential confounding factors:
 1. Up until the image around 00:38, the ice is really just a thin line on the horizon. As expected, the predictions are much better at close range.
 2. We are getting the distance to the closest part of the ice edge, not the distance to the part we are actually heading towards. The expected effect of this factor is to produce an underestimation, which is what we observe.
 3. We are relying quite a bit on the assumption that the ice moves at a constant velocity (as illustrated by the difference between the predictions with and without ice movement).

How can we do better?
 1. We can restrict our domain to the set of images actually used in the paper, taking out some of the ones where the ice is farthest away.
 2. We can try to calculate the angle to the part of the ice edge we're actually headed towards and get only this distance.
 3. I can't think of any way to address #3 other than by setting up a controlled trial (which certainly should be done in the future).

First, though, let's do a deep dive into perhaps the most informative image:

In [ ]:
import cv2
key_i = 6
key_image = images[key_i]
fig,axs = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle(fig9title(key_image))
axs[0].imshow(cv2.cvtColor(key_image["seginput"], cv2.COLOR_BGR2RGB))
s2i.plot_mask(axs[1], key_image, title="")
s2i.plot_log_polar(axs[2], key_image, "logpolar_final", title="")

The image-predicted distance to closest ice is:

In [ ]:
key_values = {}
key_values["img_closest"] = closest_ice(key_image['logpolar_final'])
print(f"{key_values['img_closest']:.2f}m")

And the image-predicted distance to farthest water (i.e., the maximum distance to ice)?

In [ ]:
def farthest_water(logpolar_plot, water_thresh=0.95, nan_thresh=0.1):
    num = np.nansum(logpolar_plot, axis=1)
    n_nonnan = (~np.isnan(logpolar_plot)).sum(axis=1)
    frac_ice = np.where(n_nonnan/logpolar_plot.shape[1] > nan_thresh, np.divide(num, n_nonnan, where=(n_nonnan != 0)), np.nan)
    frac_ice = np.fmax.accumulate(frac_ice[::-1])[::-1]  # Enforce monotonicity: if there's water behind a complete wall of ice, it doesn't count
    is_water = frac_ice <= water_thresh
    i_first = np.argmax(is_water)
    dist_first = s2i.y2dist(i_first)
    return dist_first

key_values["img_farthest"] = farthest_water(key_image['logpolar_final'])
print(f"{key_values['img_farthest']:.2f}m")

So regardless of confounding factor #2, we would expect the actual distance to ice to be within that range. However, the "actual" distance to ice, as best as we can calculate it from our position data, is:

In [ ]:
print(summary_table[["no_ice_mvt", "yes_ice_mvt"]].iloc[key_i])
key_values["no_ice_mvt"] = summary_table["no_ice_mvt"].iloc[key_i]
key_values["yes_ice_mvt"] = summary_table["yes_ice_mvt"].iloc[key_i]

Another datapoint: just eyeballing it, I'd say the maximum distance to ice looks to be roughly `60` meters.

One more sanity check: let's do a ballpark trig estimate of the ice distances. I measure the closest ice to be roughly `105` pixels down from the horizon, and the farthest water `29` pixels.

In [ ]:
key_px_down_to_dist = lambda px, height=key_image['logpolar_final'].shape[0]: \
    np.tan(np.arctan(s2i.cam_props["horizon_distance"]/s2i.cam_props["camera_height"]) -
        np.deg2rad((s2i.cam_props["vert_fov"]*s2i.cam_props["dist_factor"])*px/height))*s2i.cam_props["camera_height"]

key_values["trig_closest"] = key_px_down_to_dist(105)
key_values["trig_farthest"] = key_px_down_to_dist(29)
print(f"Closest distance: {key_values['trig_closest']:.2f}m")
print(f"Farthest distance: {key_values['trig_farthest']:.2f}m")
print(key_px_down_to_dist(images[0]["segmap"].shape[0]))
print(s2i.cam_props["near_distance"])

In [ ]:
# with pd.option_context("display.precision", 2): display(pd.DataFrame.from_dict(key_values, orient="index"))
plt.figure(figsize=(10,4))
bars = plt.bar(key_values.keys(), key_values.values())

What is going on here?
 * `yes_ice_mvt` seems to be an overestimate
 * We seem to be heading towards not the closest ice
 * The manual trig estimates are still somewhat higher than the automatic image estimates. Why?

Okay, so we still have some puzzling results, but the algorithm doesn't look *quite* so bad anymore.

What about the previous image?

In [ ]:
key2_i = 5
key2_image = images[key2_i]
fig,axs = plt.subplots(1, 3, figsize=(20, 5))
fig.suptitle(fig9title(key2_image))
axs[0].imshow(cv2.cvtColor(key2_image["seginput"], cv2.COLOR_BGR2RGB))
s2i.plot_mask(axs[1], key2_image, title="")
s2i.plot_log_polar(axs[2], key2_image, "logpolar_final", title="")
key2_values = {}
key2_values["img_closest"] = closest_ice(key2_image['logpolar_final'])
print(f"{key2_values['img_closest']:.2f}m")
key2_values["img_farthest"] = farthest_water(key2_image['logpolar_final'])
print(f"{key2_values['img_farthest']:.2f}m")
print(summary_table[["no_ice_mvt", "yes_ice_mvt"]].iloc[key2_i])
key2_values["no_ice_mvt"] = summary_table["no_ice_mvt"].iloc[key2_i]
key2_values["yes_ice_mvt"] = summary_table["yes_ice_mvt"].iloc[key2_i]
key2_values["trig_closest"] = key_px_down_to_dist(11, key2_image['logpolar_final'].shape[0])
key2_values["trig_farthest"] = key_px_down_to_dist(0)
print(f"Closest distance: {key2_values['trig_closest']:.2f}m")
print(f"Farthest distance: {key2_values['trig_farthest']:.2f}m")
plt.figure(figsize=(10,4))
bars = plt.bar(key2_values.keys(), key2_values.values())

Committing for the historical record because this is the version I used for my presentation, but when I went to clean up and commit this code I realized there's something weird going on with `farthest_water` that TODO I should investigate. In any case, we need more controlled data to really well validate the algorithm.